<br>
<pre><b>Name       :   Chaitanya Machhindra Nawale</b>
<b>Roll No.   :   BECOB215
<b>Assignment :   Vector Addition and Matrix Multiplication<b></pre>
<br>

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-12-04 05:43:43--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?HysyyD7Twt44MpC2p4fXPMxAcVgKufn-WWdA_bWDFbNLi2ZblEfzIQgUIFX482iOh2FL3bLsSojUpDKqdPXsu25ulaZuEo0Bvw3bFmJm7IIylQQw7eLldEyrXUZd0PQvhQbYn-59DulDgKjJYIU4noCzvSLPGTweueCezl46FJgXTWjANmud8OmmmoU1lfYR1SliMCc7smwkOykFe-c [following]
--2020-12-04 05:43:44--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?HysyyD7Twt44MpC2p4fXPMxAcVgKufn-WWdA_bWDFbNLi2ZblEfzIQgUIFX482iOh2FL3bLsSojUpDKqdPXsu25ulaZuEo0Bvw3bFmJm7IIylQQw7eLldEyrXUZd

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_bk3dtlj
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_bk3dtlj
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=8063fdcdab12d52efca853889e6c810b8f75d98cd61cfaf5da77bca1f9e85999
  Stored in directory: /tmp/pip-ephem-wheel-cache-evulxk5v/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define n 512

__global__ void vectAdd(int *a, int *b, int *result)
{
	int i = threadIdx.x;
	result[i] = a[i] + b[i];
}

int main()
{
	int num_blocks = 1, num_threads = n;

	int *a, *b, *c;
	int *dev_a, *dev_b, *dev_c;

	int size = n * sizeof(int);

	a = (int*)malloc(size);
	b = (int*)malloc(size);
	c = (int*)malloc(size);

	cudaMalloc((void**)&dev_a,size);
	cudaMalloc((void**)&dev_b,size);
	cudaMalloc((void**)&dev_c,size);

	for(int i = 0;i<n;i++)
	{
		//a[i] = rand()%1024;
		//b[i] = rand()%1024;
		a[i] = i;
		b[i] = i;
	}

	cudaMemcpy(dev_a,a,size,cudaMemcpyHostToDevice);
	cudaMemcpy(dev_b,b,size,cudaMemcpyHostToDevice);

	vectAdd <<<num_blocks, num_threads>>>(dev_a,dev_b,dev_c);

	cudaMemcpy(c,dev_c,size,cudaMemcpyDeviceToHost);

	for(int i = 0;i<n;i++)
		printf("%d  ",c[i]);
    
    printf("\n");
	cudaFree(dev_a);
	cudaFree(dev_b);
	cudaFree(dev_c);

	return 0;
}

0  2  4  6  8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98  100  102  104  106  108  110  112  114  116  118  120  122  124  126  128  130  132  134  136  138  140  142  144  146  148  150  152  154  156  158  160  162  164  166  168  170  172  174  176  178  180  182  184  186  188  190  192  194  196  198  200  202  204  206  208  210  212  214  216  218  220  222  224  226  228  230  232  234  236  238  240  242  244  246  248  250  252  254  256  258  260  262  264  266  268  270  272  274  276  278  280  282  284  286  288  290  292  294  296  298  300  302  304  306  308  310  312  314  316  318  320  322  324  326  328  330  332  334  336  338  340  342  344  346  348  350  352  354  356  358  360  362  364  366  368  370  372  374  376  378  380  382  384  386  388  390  392  394  396  398  400  402  404  406  408  410  412  414  416  418  420  

In [6]:
%%cu
#include<stdio.h>
#define Width 4
#define TILE_WIDTH 2
__global__ void mat_mult(int *a, int *b,int *ab, int width)
{
	// shorthand
	int tx = threadIdx.x, ty = threadIdx.y;
	int bx = blockIdx.x, by = blockIdx.y;
 
	__shared__ int s_a[TILE_WIDTH][TILE_WIDTH];
	__shared__ int s_b[TILE_WIDTH][TILE_WIDTH];

	int row = by*blockDim.y + ty;
	int col = bx*blockDim.x + tx;
	int result = 0;

	
	for(int p = 0; p < width/TILE_WIDTH; ++p)
	{
		s_a[ty][tx] = a[row*width + (p*TILE_WIDTH + tx)];
		s_b[ty][tx] = b[(p*TILE_WIDTH + ty)*width + col];
		__syncthreads();

		for(int k = 0; k < TILE_WIDTH; ++k)
		result += s_a[ty][k] * s_b[k][tx];
		__syncthreads();
	}
	ab[row*width+col] = result;
}


int main()
{
    int mat_size=Width*Width*sizeof(int);
    int tot_elements=Width*Width;
    int *M,*N,*P,*ptr;
	int a=0,x=1;
	int i=0;
	int *Md,*Nd,*Pd;
	//int size=Width*Width*sizeof(int);
	

	M=(int*)malloc(mat_size);
	N=(int*)malloc(mat_size);
	P=(int*)malloc(mat_size);
	//P_CPU=(int*)malloc(mat_size);
	ptr=M;
	printf("\nRandom Matrix A Elements Generation\n");
	for(i=0;i<tot_elements;i++)
	{	//a=(rand()%10);
		//*ptr=a;
		*ptr=x++;
		ptr++;
	}
	ptr=N;
	for(i=0;i<tot_elements;i++)
	{
		//a=(rand()%10);
		*ptr=x--;
		ptr++;
	}
	printf("\nMatrix A\n ");
	for(int i=0;i<Width*Width;i++)
	{	if(i%(Width)==0){
			printf("\n");
		}
		printf("%d ",M[i]);
	}
printf("\nMatrix B\n ");
	for(int i=0;i<Width*Width;i++)
	{	if(i%(Width)==0){
			printf("\n");
		}
		printf("%d ",N[i]);
	}
	cudaMalloc((void**)&Md,mat_size);
	cudaMemcpy(Md,M,mat_size,cudaMemcpyHostToDevice);
	cudaMalloc((void**)&Nd,mat_size);
	cudaMemcpy(Nd,N,mat_size,cudaMemcpyHostToDevice);
	cudaMalloc((void**)&Pd,mat_size);

	dim3 dimGrid(TILE_WIDTH,TILE_WIDTH);
	dim3 dimBlock(Width/TILE_WIDTH,Width/TILE_WIDTH);

	mat_mult<<<dimGrid,dimBlock>>>(Md,Nd,Pd,Width);
	

	cudaMemcpy(P,Pd,mat_size,cudaMemcpyDeviceToHost);
	
printf("\nProduct\n ");
	for(int i=0;i<Width*Width;i++)
	{	if(i%(Width)==0){
			printf("\n");
		}
		printf("%d ",P[i]);
	}
	//Free device memory
	cudaFree(Md);
	cudaFree(Nd);
	cudaFree(Pd);
free(M);
free(N);
free(P);
}


Random Matrix A Elements Generation

Matrix A
 
1 2 3 4 
5 6 7 8 
9 10 11 12 
13 14 15 16 
Matrix B
 
17 16 15 14 
13 12 11 10 
9 8 7 6 
5 4 3 2 
Product
 
90 80 70 60 
266 240 214 188 
442 400 358 316 
618 560 502 444 
